In [2]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import json
import os
import re
from tqdm.auto import tqdm

# ==========================
# 1. CONFIGURATION
# ==========================

# --- PATHS ---
CHECKPOINT_DIR = "checkpoints"
MODEL_FILE = "model.pth" # Or "autoencoder_epoch_X.pth"
VOCAB_FILE = os.path.join(CHECKPOINT_DIR, "vocab_full.json")

# --- TEST FILE (CHANGE THIS!) ---
# Point this to the text file you want to test (ex: a new log file)
TEST_FILE_PATH = r"data/kafka/log-2018-06-01/129546168_2018-05-31-23-37-13.txt"
# If you want to test the training set, put a path here instead:
# TEST_FILE_PATH = "data/kafka/log-2018-06-01/129546169_2018-05-31-23-46-03.txt"

# --- CONFIG ---
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
MAX_LEN = 128
BATCH_SIZE = 128

# Regex Patterns (Same as training)
PATTERNS_PY = {
    r"\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}\.\d{3}Z": "[TIMESTAMP_ISO]",
    r"\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}": "[TIMESTAMP]",
    r"PID\s+\d+": "PID [PID_NUM]",
    r"\[\d+\]": "[ID]",
    r"(?:\d{1,3}\.){3}\d{1,3}": "[IP_ADDR]",
    r"\s+": " "
}

In [9]:
# ==========================
# 2. MODEL DEFINITION (Must be identical to training)
# ==========================

class LogAutoEncoder(nn.Module):
    def __init__(self, vocab_size, embed_dim=48, latent_dim=24, max_len=128):
        super(LogAutoEncoder, self).__init__()
        
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.encoder = nn.LSTM(embed_dim, latent_dim, batch_first=True, num_layers=1)
        self.decoder = nn.LSTM(latent_dim, embed_dim, batch_first=True, num_layers=1)
        self.output_linear = nn.Linear(embed_dim, vocab_size)
        self.max_len = max_len
        self.embed_dim = embed_dim
        self.latent_dim = latent_dim

    def forward(self, x):
        embedded = self.embedding(x)
        _, (h_n, c_n) = self.encoder(embedded)
        latent = h_n.squeeze(0)
        latent_repeated = latent.unsqueeze(1).expand(-1, self.max_len, -1)
        # Decoder expects hidden/state of size (num_layers, batch, embed_dim)
        batch_size = x.size(0)
        h0 = torch.zeros(1, batch_size, self.embed_dim, device=x.device)
        c0 = torch.zeros(1, batch_size, self.embed_dim, device=x.device)
        decoded, _ = self.decoder(latent_repeated, (h0, c0))
        reconstruction = self.output_linear(decoded)
        return reconstruction, latent

In [10]:
# ==========================
# 3. HELPER FUNCTIONS
# ==========================

def clean_text(text):
    """Apply regex patterns to clean a single log."""
    if not isinstance(text, str): return ""
    for pat, repl in PATTERNS_PY.items():
        text = re.sub(pat, repl, text)
    return text

def text_to_ids(text, vocab, unk_id, pad_id, max_len):
    """Convert text string to padded IDs."""
    if not isinstance(text, str): return [pad_id] * max_len
    tokens = text.split()
    ids = [vocab.get(word, unk_id) for word in tokens]
    
    # Truncate or Pad
    if len(ids) > max_len:
        ids = ids[:max_len]
    else:
        ids = ids + [pad_id] * (max_len - len(ids))
    
    return ids

def calculate_reconstruction_loss(model, input_ids, vocab_size, criterion, device):
    """Pass data through model and calculate loss (Anomaly Score)."""
    model.eval()
    
    with torch.no_grad():
        # Forward pass
        input_tensor = torch.tensor(input_ids, dtype=torch.long).unsqueeze(0).to(device)
        reconstruction, latent = model(input_tensor)
        
        # Calculate loss
        input_flat = input_tensor.view(-1)
        reconstruction_flat = reconstruction.view(-1, vocab_size)
        
        loss = criterion(reconstruction_flat, input_flat)
        
    return loss.item(), latent

In [ ]:
# ==========================
# 3.5 DRAIN ALGORITHM (Simplified)
# ==========================

class SimpleDrain:
    """
    Simplified Drain algorithm for log template extraction.
    Groups logs by length, then finds similar patterns to extract templates.
    """
    def __init__(self, similarity_threshold=0.5, max_depth=4):
        self.similarity_threshold = similarity_threshold
        self.max_depth = max_depth
        self.templates = {}  # template_id -> template_tokens
        self.template_counter = 0
        
    def tokenize(self, log_text):
        """Split log into tokens."""
        return log_text.split()
    
    def get_similarity(self, tokens1, tokens2):
        """Calculate similarity between two token sequences."""
        if len(tokens1) != len(tokens2):
            return 0.0
        matches = sum(1 for t1, t2 in zip(tokens1, tokens2) if t1 == t2)
        return matches / len(tokens1) if len(tokens1) > 0 else 0.0
    
    def find_best_template(self, tokens):
        """Find the best matching template for given tokens."""
        best_template_id = None
        best_similarity = 0.0
        
        # Filter templates by same length
        token_len = len(tokens)
        for template_id, template_tokens in self.templates.items():
            if len(template_tokens) == token_len:
                similarity = self.get_similarity(tokens, template_tokens)
                if similarity >= self.similarity_threshold and similarity > best_similarity:
                    best_similarity = similarity
                    best_template_id = template_id
        
        return best_template_id, best_similarity
    
    def create_template(self, tokens1, tokens2):
        """Merge two token sequences into a template with wildcards."""
        template = []
        for t1, t2 in zip(tokens1, tokens2):
            if t1 == t2:
                template.append(t1)
            else:
                template.append("<*>")  # Wildcard for variable parts
        return template
    
    def extract_parameters(self, tokens, template_tokens):
        """Extract variable parameters from tokens using template."""
        parameters = []
        for token, template_token in zip(tokens, template_tokens):
            if template_token == "<*>":
                parameters.append(token)
        return parameters
    
    def parse(self, log_text):
        """
        Parse a log line and return (template_id, template_str, parameters).
        """
        tokens = self.tokenize(log_text)
        
        if not tokens:
            return None, "", []
        
        # Find best matching template
        template_id, similarity = self.find_best_template(tokens)
        
        if template_id is not None:
            # Use existing template
            template_tokens = self.templates[template_id]
            parameters = self.extract_parameters(tokens, template_tokens)
            template_str = " ".join(template_tokens)
            return template_id, template_str, parameters
        else:
            # Create new template
            self.template_counter += 1
            template_id = self.template_counter
            
            # First occurrence: template = exact tokens
            self.templates[template_id] = tokens.copy()
            template_str = " ".join(tokens)
            parameters = []  # No wildcards yet
            
            return template_id, template_str, parameters
    
    def update_template(self, template_id, new_tokens):
        """Update an existing template by merging with new tokens."""
        if template_id in self.templates:
            old_template = self.templates[template_id]
            merged_template = self.create_template(old_template, new_tokens)
            self.templates[template_id] = merged_template

# Initialize Drain parser
drain_parser = SimpleDrain(similarity_threshold=0.5, max_depth=4)

print("✅ Drain parser initialized")

In [11]:
# ==========================
# 4. LOADING MODEL & VOCAB
# ==========================

print(f"🤖 Loading Model: {MODEL_FILE}")
print(f"📚 Loading Vocab: {VOCAB_FILE}")

# Load Vocab
with open(VOCAB_FILE, 'r') as f:
    vocab_data = json.load(f)

vocab = vocab_data['vocab']
vocab_size = vocab_data['size']
unk_id = vocab[vocab_data['unk_token']]
pad_id = vocab[vocab_data['pad_token']]

print(f"✅ Vocab loaded. Size: {vocab_size}")

# Load Model
model = LogAutoEncoder(vocab_size=vocab_size).to(DEVICE)

# Load Checkpoint
try:
    checkpoint = torch.load(MODEL_FILE, map_location=DEVICE)
    
    # Check if checkpoint has 'model_state_dict' or is just weights
    if 'model_state_dict' in checkpoint:
        model.load_state_dict(checkpoint['model_state_dict'])
        print(f"✅ Model weights loaded from checkpoint.")
    else:
        model.load_state_dict(checkpoint)
        print(f"✅ Model weights loaded (direct).")
        
except FileNotFoundError:
    print(f"❌ Model file not found: {MODEL_FILE}")
    exit()

# Setup Loss function (Must match training: ignore padding)
criterion = nn.CrossEntropyLoss(ignore_index=pad_id)

🤖 Loading Model: model.pth
📚 Loading Vocab: checkpoints\vocab_full.json
✅ Vocab loaded. Size: 50000
✅ Model weights loaded (direct).


In [ ]:
# Quick single-log test (no file read) - WITH DRAIN
SAMPLE_LOG = """16:37:15     INFO - MultiFileLogger online at 20180531 16:37:15 in /builds/slave/test\n16:37:15     INFO - Run as scripts/scripts/desktop_unittest.py --cfg unittests/mac_unittest.py --mochitest-suite browser-chrome-chunked --e10s --total-chunks 7 --this-chunk 5 --blob-upload-branch mozilla-esr52 --download-symbols true\n16:37:15     INFO - Dumping config to /builds/slave/test/logs/localconfig.json.\n16:37:15     INFO - {'all_cppunittest_suites': {'cppunittest': ('tests/cppunittest',)},\n16:37:15     INFO -  'all_gtest_suites': {'gtest': ()},\n16:37:15     INFO -  'all_jittest_suites': {'jittest': ()},\n16:37:15     INFO -  'all_mochitest_suites': {'a11y': ('--flavor=a11y',),\n16:37:15     INFO -                           'browser-chrome': ('--flavor=browser',),\n16:37:15     INFO -                           'browser-chrome-addons': ('--flavor=browser',\n16:37:15     INFO -                                                     '--chunk-by-runtime',\n16:37:15     INFO -                                                     '--tag=addons'),\n16:37:15     INFO -                           'browser-chrome-chunked': ('--flavor=browser',\n16:37:15     INFO -                                                      '--chunk-by-runtime'),\n16:37:15     INFO -                           'browser-chrome-clipboard': ('--flavor=browser',\n16:37:15     INFO -                                                        '--subsuite=clipboard'),\n16:37:15     INFO -                           'browser-chrome-gpu': ('--flavor=browser',\n16:37:15     INFO -                                                  '--subsuite=gpu'),\n16:37:15     INFO -                           'browser-chrome-screenshots': ('--flavor=browser',\n16:37:15     INFO -                                                          '--subsuite=screenshots'),\n16:37:15     INFO -                           'chrome': ('--flavor=chrome',),\n16:37:15     INFO -                           'chrome-chunked': ('--flavor=chrome',\n16:37:15     INFO -                                              '--chunk-by-dir=4'),\n16:37:15     INFO -                           'chrome-clipboard': ('--flavor=chrome',\n16:37:15     INFO -                                                '--subsuite=clipboard'),\n16:37:15     INFO -                           'chrome-gpu': ('--flavor=chrome', '--subsuite=gpu'),\n16:37:15     INFO -                           'jetpack-addon': ('--flavor=jetpack-addon',),\n16:37:15     INFO -                           'jetpack-package': ('--flavor=jetpack-package',),\n16:37:15     INFO -                           'jetpack-package-clipboard': ('--flavor=jetpack-package',\n16:37:15     INFO -                                                         '--subsuite=clipboard'),\n16:37:15     INFO -                           'mochitest-devtools-chrome': ('--flavor=browser',\n16:37:15     INFO -                                                         '--subsuite=devtools'),\n16:37:15     INFO -                           'mochitest-devtools-chrome-chunked': ('--flavor=browser',\n16:37:15     INFO -                                                                 '--subsuite=devtools',\n16:37:15     INFO -                                                                 '--chunk-by-runtime'),\n16:37:15     INFO -                           'mochitest-gl': ('--subsuite=webgl',),\n16:37:15     INFO -                           'mochitest-media': ('--subsuite=media',),\n16:37:15     INFO -                           'plain': (),\n16:37:15     INFO -                           'plain-chunked': ('--chunk-by-dir=4',),\n16:37:15     INFO -                           'plain-clipboard': ('--subsuite=clipboard',),\n16:37:15     INFO -                           'plain-gpu': ('--subsuite=gpu',)},\n16:37:15     INFO -  'all_mozbase_suites': {'mozbase': ()},\n16:37:15     INFO -  'all_reftest_suites': {'crashtest': {'options': ('--suite=crashtest',),\n16:37:15     INFO -                                       'tests': ('tests/reftest/tests/testing/crashtest/crashtests.list',)},\n16:37:15     INFO -                         'jsreftest': {'options': ('--extra-profile-file=tests/jsreftest/tests/user.js',),\n16:37:15     INFO -                                       'tests': ('tests/jsreftest/tests/jstests.list',)},\n16:37:15     INFO -                         'reftest': {'options': ('--suite=reftest',),\n16:37:15     INFO -                                     'tests': ('tests/reftest/tests/layout/reftests/reftest.list',)}},\n16:37:15     INFO -  'all_xpcshell_suites': {'xpcshell': {'options': ('--xpcshell=%(abs_app_dir)s/xpcshell',\n16:37:15     INFO -                                                   '--manifest=tests/xpcshell/tests/xpcshell.ini'),\n16:37:15     INFO -                                       'tests': ()},\n16:37:15     INFO -                          'xpcshell-addons': {'options': ('--xpcshell=%(abs_app_dir)s/xpcshell',\n16:37:15     INFO -                                                          '--tag=addons',\n16:37:15     INFO -                                                          '--manifest=tests/xpcshell/tests/xpcshell.ini'),\n16:37:15     INFO -                                              'tests': ()}},\n16:37:15     INFO -  'allow_software_gl_layers': False,\n16:37:15     INFO -  'append_to_log': False,\n16:37:15     INFO -  'base_work_dir': '/builds/slave/test',\n16:37:15     INFO -  'blob_upload_branch': 'mozilla-esr52',\n16:37:15     INFO -  'blob_uploader_auth_file': '/builds/slave/test/oauth.txt',\n16:37:15     INFO -  'buildbot_json_path': 'buildprops.json',\n16:37:15     INFO -  'buildbot_max_log_size': 52428800,\n16:37:15     INFO -  'code_coverage': False"""

model.eval()
with torch.no_grad():
    # Step 1: Clean
    cleaned = clean_text(SAMPLE_LOG)
    
    # Step 2: DRAIN - Extract template and parameters
    template_id, template_str, parameters = drain_parser.parse(cleaned)
    
    # Step 3: Encode (use template instead of full cleaned log)
    ids = text_to_ids(template_str, vocab, unk_id, pad_id, MAX_LEN)
    
    # Step 4: Calculate anomaly score
    loss, latent = calculate_reconstruction_loss(model, ids, vocab_size, criterion, DEVICE)

print("Single-log test result (with Drain):")
print(f"Raw (first 120 chars): {SAMPLE_LOG[:120]}...")
print(f"Cleaned (first 120 chars): {cleaned[:120]}...")
print(f"Template ID: {template_id}")
print(f"Template (first 120 chars): {template_str[:120]}...")
print(f"Parameters extracted: {parameters[:5]}..." if len(parameters) > 5 else f"Parameters: {parameters}")
print(f"Anomaly score: {loss:.4f}")

Single-log test result:
Raw (first 120 chars): 16:37:15     INFO - MultiFileLogger online at 20180531 16:37:15 in /builds/slave/test
16:37:15     INFO - Run as scripts...
Cleaned (first 120 chars): 16:37:15 INFO - MultiFileLogger online at 20180531 16:37:15 in /builds/slave/test 16:37:15 INFO - Run as scripts/scripts...
Anomaly score: 11.4067


In [ ]:
# ==========================
# 5. RUNNING THE TEST - WITH DRAIN
# ==========================

# Ensure file exists
if not os.path.exists(TEST_FILE_PATH):
    print(f"❌ Test file not found: {TEST_FILE_PATH}")
    # Fallback: Try listing directory
    print(f"   Looking in: {os.path.dirname(TEST_FILE_PATH)}")
    if os.path.exists(os.path.dirname(TEST_FILE_PATH)):
        files = os.listdir(os.path.dirname(TEST_FILE_PATH))
        print(f"   Files found: {files[:5]}")
    exit()

# Process file
results = []

print(f"📂 Reading file: {TEST_FILE_PATH}")
print(f"🔄 Processing with Drain & Calculating Anomaly Scores...")

# We read line by line to avoid OOM
try:
    with open(TEST_FILE_PATH, 'r', encoding='utf-8', errors='ignore') as f:
        for line in tqdm(f, desc="Testing Logs"):
            if not line.strip(): continue
            
            # 1. Clean
            cleaned = clean_text(line)
            
            # 2. DRAIN - Extract template and parameters
            template_id, template_str, parameters = drain_parser.parse(cleaned)
            
            # 3. Encode (use template for anomaly detection)
            ids = text_to_ids(template_str, vocab, unk_id, pad_id, MAX_LEN)
            
            # 4. Predict
            loss, latent = calculate_reconstruction_loss(model, ids, vocab_size, criterion, DEVICE)
            
            results.append({
                'raw_log': line,
                'cleaned_log': cleaned,
                'template_id': template_id,
                'template': template_str,
                'parameters': str(parameters),  # Convert list to string for CSV
                'num_parameters': len(parameters),
                'anomaly_score': loss,
                'latent_vector': latent.cpu().numpy().tolist()
            })

except Exception as e:
    print(f"❌ Error processing file: {e}")
    exit()

print(f"\n✅ Processed {len(results)} log lines")
print(f"🔍 Discovered {len(drain_parser.templates)} unique templates")

📂 Reading file: data/kafka/log-2018-06-01/129546168_2018-05-31-23-37-13.txt
🔄 Processing & Calculating Anomaly Scores...


Testing Logs: 0it [00:00, ?it/s]

In [ ]:
# ==========================
# 5.5 DRAIN TEMPLATES ANALYSIS
# ==========================

print(f"\n📋 DRAIN TEMPLATE ANALYSIS")
print(f"="*70)
print(f"Total unique templates discovered: {len(drain_parser.templates)}")
print(f"="*70)

# Show top 10 most common templates
from collections import Counter
template_counts = Counter([r['template_id'] for r in results])

print(f"\nTop 10 Most Frequent Templates:")
print("-" * 70)
for template_id, count in template_counts.most_common(10):
    template = drain_parser.templates.get(template_id, [])
    template_str = " ".join(template)
    # Truncate long templates
    display_template = template_str[:80] + "..." if len(template_str) > 80 else template_str
    print(f"Template {template_id:3d} | Count: {count:5d} | {display_template}")
print("-" * 70)

In [ ]:
# ==========================
# 6. ANALYZE & VISUALIZATION
# ==========================

# Convert to Pandas for analysis
df_results = pd.DataFrame(results)

# Calculate Statistics
mean_score = df_results['anomaly_score'].mean()
std_score = df_results['anomaly_score'].std()
max_score = df_results['anomaly_score'].max()

# Define Threshold (Mean + 3*Std is a common heuristic)
anomaly_threshold = mean_score + 3 * std_score

print("\n" + "="*70)
print(f"📊 ANOMALY SCORES SUMMARY (Template-based)")
print(f"="*70)
print(f"Mean Score:          {mean_score:.4f}")
print(f"Std Deviation:       {std_score:.4f}")
print(f"Max Score:           {max_score:.4f}")
print(f"Threshold (M+3*SD):  {anomaly_threshold:.4f}")
print(f"="*70)

# Highlight Anomalies (Logs above threshold)
anomalies = df_results[df_results['anomaly_score'] > anomaly_threshold]

print(f"\n🚨 Found {len(anomalies)} Anomalies (Score > {anomaly_threshold:.4f})")
print(f"="*70)

# Show top 5 most anomalous logs with their templates
print("Top 5 Most Anomalous Logs:")
print("-" * 70)
for idx, row in anomalies.nlargest(5, 'anomaly_score').iterrows():
    print(f"Score: {row['anomaly_score']:.4f} | Template ID: {row['template_id']}")
    print(f"  Template: {row['template'][:100]}...")
    print(f"  Params: {row['parameters'][:80]}")
    print(f"  Raw: {row['raw_log'][:80]}")
    print("-" * 70)

# Analyze templates in anomalies
print(f"\n📋 Template Distribution in Anomalies:")
template_in_anomalies = anomalies['template_id'].value_counts()
print(f"Unique templates in anomalies: {len(template_in_anomalies)}")
if len(template_in_anomalies) > 0:
    print(f"Most common anomalous template ID: {template_in_anomalies.index[0]} ({template_in_anomalies.iloc[0]} occurrences)")

# Save results
output_csv = f"{CHECKPOINT_DIR}/test_anomaly_results.csv"
df_results.to_csv(output_csv, index=False)
print(f"\n💾 Results saved to: {output_csv}")

# Save anomalies separately
anomalies_csv = f"{CHECKPOINT_DIR}/anomalies_only.csv"
anomalies.to_csv(anomalies_csv, index=False)
print(f"💾 Anomalies saved to: {anomalies_csv}")

# Save templates
templates_output = f"{CHECKPOINT_DIR}/drain_templates.json"
templates_dict = {
    str(tid): " ".join(tokens) 
    for tid, tokens in drain_parser.templates.items()
}
with open(templates_output, 'w') as f:
    json.dump(templates_dict, f, indent=2)
print(f"💾 Templates saved to: {templates_output}")


📊 ANOMALY SCORES SUMMARY
Mean Score:    3.2490
Std Deviation:   1.4185
Max Score:      14.0555
Threshold (M+3*SD): 7.5045

🚨 Found 318 Anomalies (Score > 7.5045)
Top 5 Most Anomalous Logs:
--------------------------------------------------
Score: 14.0555 | Log: Saving to: 'archiver_client.py'

Score: 11.8500 | Log: Connecting to hg.mozilla.org|63.245.215.102|:443... connected.

Score: 11.6276 | Log: rm -f oauth.txt

Score: 11.6031 | Log: rm -rf scripts properties

Score: 11.3854 | Log:   PROPERTIES_FILE=/builds/slave/test/buildprops.json

--------------------------------------------------

💾 Results saved to: checkpoints/test_anomaly_results.csv
💾 Anomalies saved to: checkpoints/anomalies_only.csv
